In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [2]:
import tensorflow as tf
import numpy as np

tf.random.set_seed(42)
np.random.seed(42)

In [3]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dir = 'train-directory'
val_dir = 'validation-directory'

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)#shuffle:just dogs and cats a mix
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False) 


In [4]:
print(f"Number of training set images: {len(train_dataset)}")
print(f"Number of Validation set images: {len(val_dataset)}")

Number of training set images: 300
Number of Validation set images: 60


Modifying ResNet Models
This class modifies the pre-trained ResNet-18 model for use in specific binary classification problems (e.g. left-right hand recognition)

1 models.resnet18(pretrained=True) loads a pre-trained ResNet-18 model.

2 The "param" loop iterates over all the parameters of the ResNet-18 model and sets their requires_grad attribute to False. this means that they will not be updated (i.e., the layers are frozen) for the rest of the training process.

3 Replacing the fully connected layer
original_resnet.fc.in_features Get the number of input features for the original fully connected layer (classification layer).
original_resnet.fc = nn.Linear(num_ftrs, num_classes) Replace the original fully connected layer. The new fully-connected layer has num_ftrs input features (same as the original layer) and num_classes outputs (2 in this example) to accommodate the new classification task (left- or right-handed). Here nn.Linear is a linear transform layer used to generate the final classification result from the feature extraction output of ResNet.

In [77]:
class ModifiedResNet(nn.Module):
    def __init__(self):
        super(ModifiedResNet, self).__init__()
        original_resnet = models.resnet18(pretrained=True)
        
        for param in original_resnet.parameters():
            param.requires_grad = False

        
        num_ftrs = original_resnet.fc.in_features
        original_resnet.fc = nn.Linear(num_ftrs, 2)
        self.resnet = original_resnet

    def forward(self, x):
        return self.resnet(x)


Train and visualize the model
model: The neural network model to be trained.
train_loader: Data loader for training, contains the training dataset.
val_loader: Data loader for validation, contains the validation dataset.
optimizer: optimizer used to update the weights of the model.
loss_fn: loss function, used to calculate the error between the predicted results and the actual labels.
epochs: number of training rounds, default is 10.


Training Loop
Enter the loop for each epoch by first setting the model in training mode.
Initialize the accumulated losses, the number of correct predictions, and the total number of samples processed.

Process each batch of data
Calculate model output and loss.
Clear old gradients, perform backpropagation and gradient update.
Accumulate loss and accuracy correlation calculations.

Calculate average loss and accuracy, store to list.
Print the training loss and accuracy for the current epoch.


Validation Process
Set the model to evaluation mode and disable certain layers specific to training such as Dropout.
Perform forward propagation without calculating gradients and calculate the validation loss and accuracy.

Calculate and store the average validation loss and accuracy.
Print the validation results for the current epoch.

In [78]:
def train_model(model, train_loader, val_loader, optimizer, loss_fn, epochs=10):
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        model.train()
        total_train_loss, total_train_correct, total_train = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train_correct += (predicted == labels).sum().item()
            total_train += labels.size(0)
        
        train_losses.append(total_train_loss / total_train)
        train_accuracies.append(100 * total_train_correct / total_train)
        print(f'Epoch {epoch+1}, Train Loss: {train_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.2f}%')

        model.eval()
        total_val_loss, total_val_correct, total_val = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = loss_fn(outputs, labels)
                total_val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total_val_correct += (predicted == labels).sum().item()
                total_val += labels.size(0)
        
        val_losses.append(total_val_loss / total_val)
        val_accuracies.append(100 * total_val_correct / total_val)
        print(f'Epoch {epoch+1}, Val Loss: {val_losses[-1]:.4f}, Val Accuracy: {val_accuracies[-1]:.2f}%')

    # 绘制损失和准确率图表
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
    plt.plot(range(1, epochs+1), val_losses, label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs+1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1, epochs+1), val_accuracies, label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModifiedResNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
train_model(model, train_loader, val_loader, optimizer, loss_fn, epochs=10)


Epoch 1, Train Loss: 0.7434, Train Accuracy: 53.67%
Epoch 1, Val Loss: 0.4910, Val Accuracy: 88.33%
Epoch 2, Train Loss: 0.5651, Train Accuracy: 73.67%
Epoch 2, Val Loss: 0.3458, Val Accuracy: 95.00%
Epoch 3, Train Loss: 0.4998, Train Accuracy: 78.33%
Epoch 3, Val Loss: 0.2519, Val Accuracy: 100.00%
Epoch 4, Train Loss: 0.4264, Train Accuracy: 86.00%
Epoch 4, Val Loss: 0.2855, Val Accuracy: 88.33%
Epoch 5, Train Loss: 0.4086, Train Accuracy: 82.67%
Epoch 5, Val Loss: 0.1908, Val Accuracy: 100.00%
Epoch 6, Train Loss: 0.3689, Train Accuracy: 85.00%
Epoch 6, Val Loss: 0.2611, Val Accuracy: 88.33%
Epoch 7, Train Loss: 0.3935, Train Accuracy: 81.67%
Epoch 7, Val Loss: 0.1526, Val Accuracy: 98.33%
Epoch 8, Train Loss: 0.3734, Train Accuracy: 81.67%
Epoch 8, Val Loss: 0.1608, Val Accuracy: 98.33%
Epoch 9, Train Loss: 0.3300, Train Accuracy: 86.67%
Epoch 9, Val Loss: 0.1070, Val Accuracy: 100.00%
Epoch 10, Train Loss: 0.3169, Train Accuracy: 89.00%
Epoch 10, Val Loss: 0.1105, Val Accuracy: 10